## 背景介绍

利用Google Finance API抓取了每分钟的标准普尔500指数。除了标准普尔500指数以外，还收集了其对应的500家公司的股价。在得到了这些数据之后，基于标准普尔指数观察的500家公司的股价，用深度学习模型来预测标准普尔500指数。

注意：本文只是基于TensorFlow的一个实战教程。真正预测股价是非常具有挑战性的，尤其在分钟级这样频率较高的预测中，要考虑的因素的量是庞大的。

## 导入数据集

抓取到的股票数据从爬虫服务器上导出为CSV格式的文件。该数据集包含了从2017年四月到八月共计n=41266分钟的标准普尔500指数以及500家公司的股价。

In [1]:
import pandas as pd
import numpy as np

In [2]:
# 导入数据
df = pd.read_csv('data_stocks.csv')
# 移除日期列
df = df.drop(['DATE'], 1)
# 数据集的维度
n = df.shape[0]
p = df.shape[1]
# 将数据集转化为numpy数组
data = df.values

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41266 entries, 0 to 41265
Columns: 501 entries, SP500 to NYSE.ZTS
dtypes: float64(501)
memory usage: 157.7 MB


In [4]:
df.columns

Index(['SP500', 'NASDAQ.AAL', 'NASDAQ.AAPL', 'NASDAQ.ADBE', 'NASDAQ.ADI',
       'NASDAQ.ADP', 'NASDAQ.ADSK', 'NASDAQ.AKAM', 'NASDAQ.ALXN',
       'NASDAQ.AMAT',
       ...
       'NYSE.WYN', 'NYSE.XEC', 'NYSE.XEL', 'NYSE.XL', 'NYSE.XOM', 'NYSE.XRX',
       'NYSE.XYL', 'NYSE.YUM', 'NYSE.ZBH', 'NYSE.ZTS'],
      dtype='object', length=501)

In [5]:
import matplotlib.pyplot as plt
%matplotlib inline

可以通过pyplot.plot('SP500')来快速查看标准普尔500指数的时间序列。

标准普尔500指数的时间序列图
    
注意：这里展示的是标普500指数的领先(lead)，也就是说其值是原始值在时间轴上后移一分钟得到的。因为我们要预测的是下一分钟的指数而不是当前的指数，所以这一操作是必不可少的。

## 准备训练集和测试集数据

数据集被分成训练集和测试集。训练数据为总数据集的80％。数据不进行打乱，而是按顺序切片。训练数据可以从2017年4月选取到2017年7月底，测试数据则选取到2017年8月底为止。

In [6]:
# 划分训练集和测试集
train_start = 0
train_end = int(np.floor(0.8*n))
test_start = train_end + 1
test_end = n
data_train = data[np.arange(train_start, train_end), :]
data_test = data[np.arange(test_start, test_end), :]

时间序列的交叉验证方法有很多，像有无refitting或其他像time series bootstrap resampling的精细概念的滚动预测（rolling forecasts）。后者（time series bootstrap resampling）中的重复样本是考虑时间序列的周期性分解的结果，这是为了使模拟采样同样具有周期性的特征而不是单单复制采样值。

## 数据缩放

大多数的神经网络都受益于输入值的缩放（有时也有输出值）。为什么呢？

因为大多数神经网络的激励函数都是定义在0, 1区间或-1, 1区间，像sigmoid函数和tanh函数一样。

虽然如今线性整流单元已经被广泛引用于无界的激活值问题中，但是我们还是选择将输入输出值做统一的缩放。

缩放操作可以通过sklearn中的MinMaxScaler轻松实现。

In [7]:
# 数据缩放
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(data_train)
data_train = scaler.transform(data_train)

scaler_test = MinMaxScaler()
scaler_test.fit(data_test)
data_test = scaler.transform(data_test)

In [31]:
X = data_train[:, 1:]
Y = data_train[:, 0]
Y.shape = (8253,1)
print(X.shape, Y.shape)

X_test = data_test[:, 1:]
Y_test = data_test[:, 0]
Y_test.shape = (8253,1)

(8253, 500) (8253, 1)


备注：应当仔细考虑好什么数据要在什么时候被缩放。一个常见的错误是在训练集和测试集划分前进行特征缩放。为什么这样做是错误的呢？

因为缩放的计算需要调用数据的统计值（像数据的最大最小值）。当你在真实生活中进行预测时你并没有来自未来的观测信息，所以相应地，训练数据特征缩放所用的统计值应当来源于训练集，测试集也一样。否则，在预测时使用了包含未来信息往往会导致性能指标向好的方向偏移。

## 神经网络流程

    1数据集获取（有监督数据整理）
    2神经网络参数确定，有多少层，多少个节点，激活函数是什么，损失函数是什么
    3数据预处理，标准化，中心化，特征压缩，异常值处理
    4初始化网络权重
    5网络训练
            5.1正向传播
            5.2计算loss
            5.3计算反向梯度
            5.4更新梯度
            5.5重新正向传播

In [8]:
# 引入 TensorFlow
import tensorflow as tf

### 占位符

所有的过程都从占位符开始。为了拟合模型，我们需要定义两个占位符：X包含模型输入（在T = t时刻500个成员公司的股价），Y为模型输出（T = t + 1时刻的标普指数）。

占位符的shape分别为[None, n_stocks]和[None]，意味着输入为二维矩阵，输出为一维向量。设计出恰当的神经网络的必要条件之一就是清楚神经网络需要的输入和输出维度。

In [22]:
# 占位符
n_stack = 500
x = tf.placeholder(tf.float32, [None, n_stack])
y = tf.placeholder(tf.float32, [None, 1])

None值代表着我们当前不知道每个批次中流经神经网络的观测值数量，所以为了保持该量的弹性，我们用None来填充。

稍后我们将定义控制每个批次中观测样本数量的变量batch_size。

### 变量

除了占位符，TensorFlow中的另一个基本概念是变量。

占位符在图中用来存储输入数据和输出数据，变量在图的执行过程中可以变化，是一个弹性的容器。

为了在训练中调整权重和偏置，它们被定义为变量。变量需要在训练开始前进行初始化。变量的初始化稍后我们会单独讲解。

我们的模型包含四个层。第一层有1024个神经元，比输入变量的两倍还要多一点。紧接在后面的隐藏层是前面一层的一半，即后面层的神经元个数分别为512,256和128。每层中神经元数量的减少也意味着信息量的压缩。当然还有其他的神经网络结构，但是不在本文的讨论范围当中。

In [10]:
#??tf.random_normal

In [21]:
# 权重和偏置项初始化
def weight_init(shape):
    return tf.random_normal(shape)

def bias_init(shape):
    return tf.random_normal(shape)

In [23]:
# 模型结构参数
n1 = 1024
n2 = 512
n3 = 256
n4 = 128
n_target = 1

# 第一层 : 隐藏层权重和偏置变量
w1 = tf.Variable(weight_init([n_stack, n1]))
b1 = tf.Variable(bias_init([n1]))

# 第二层 : 隐藏层权重和偏置变量
w2 = tf.Variable(weight_init([n1, n2]))
b2 = tf.Variable(bias_init([n2]))

# 第三层: 隐藏层权重和偏置变量
w3 = tf.Variable(weight_init([n2, n3]))
b3 = tf.Variable(bias_init([n3]))

# 第四层: 隐藏层权重和偏置变量
w4 = tf.Variable(weight_init([n3, n4]))
b4 = tf.Variable(bias_init([n4]))

# 输出层: 输出权重和偏置变量
w_out = tf.Variable(weight_init([n4, n_target]))
b_out = tf.Variable(bias_init([n_target]))

清楚输入层、隐藏层和输出层的变量对应的维度是非常重要的。

在多层感知机的经验法则中（MLPs，本文就是按照该准则设计的网络），前一层权重的维度数组中的第二个元素与当前层中权重维度数组的第一个元素数值相等。听起来可能有些复杂，但是为了使当前层的输入作为输入传入下一层，这样的法则是必要的。偏置的维度等于当前层权重维度数组中的第二个元素，对应当前层中神经元的数量。

### 设计网络架构

在定义了所需的权重和偏置变量之后，网络的拓扑结构即网络的架构需要被确定下来。在TensorFlow中，即需要将占位符（数据）和变量（权重和偏置）整合入矩阵乘法的序列当中。

除此之外，神经网络中是经过了激活函数的转换的。激活函数是神经网络架构中非常的元素之一，在非线性系统中尤其如此。目前已经有很多中可供使用的激活函数，本文中的模型选用了最常用的整流线性单元（ReLU）。

In [24]:
# 隐藏层
L1 = tf.nn.relu(tf.matmul(x, w1) + b1)
L2 = tf.nn.relu(tf.matmul(L1, w2) + b2)
L3 = tf.nn.relu(tf.matmul(L2, w3) + b3)
L4 = tf.nn.relu(tf.matmul(L3, w4) + b4)

# 输出层 (必须经过转置)
out = tf.transpose(tf.matmul(L4, w_out) + b_out)

图形说明了网络架构。模型一共包含了三个主要的组件：输入层，隐藏层和输出层。图示的结构被称为前馈网络，前馈意味着从左侧输入的数据将径自向右传播。与之相对的网络结构如recurrent neural networks（RNN）允许数据流在网络结构中反向传播。

### 损失函数

网络的损失函数可以根据网络的预测值和训练集中的实际观测值来生成度量偏差程度的指标。

在回归问题当中，最常用的损失函数为均方误差（MSE）。均方误差计算的就是预测值和目标值的误差平方值的平均值。基本上任何可微函数都可以用于计算预测值和目标值之间的偏差程度。

In [25]:
# 损失函数   tf.square(out-Y)
loss = tf.reduce_mean(tf.square(out - y))

### 优化器

优化器负责训练过程中调整网络的权重和偏置的关键操作。这些操作中包含着梯度运算，梯度方向对应的就是训练过程中最小化网络损失函数的方向。稳定而又高效的优化器是神经网络中深入研究的课题之一。

In [26]:
# 优化器
train = tf.train.AdamOptimizer(0.01).minimize(loss)

这里我们使用Adam优化器，目前它是深度学习中默认的优化器。Adam的全称为Adaptive Moment Estimation，可以视为其他两个优化器AdaGrad和RMSProp的结合。

### 拟合神经网络

在定义了网络的占位符，变量，初始化器，损失函数和优化器之后，模型需要进入正式的训练过程。通常我们使用minibatch的方式进行训练（小的batch size）。在这种训练方式中，我们从训练集中随机抽取n = sample_size的数据样本送入网络进行训练。训练集被划分为n / batch_size个批次并按顺序送入网络。这时占位符X和Y参与了这一过程，它们分别存储输入值和目标值并作为输入和目标送入网络。

样本数据X将在网络中传播直至输出层。到达输出层后，TensorFlow将把模型的当前预测值与当前批次的实际观测值Y进行比较。随后，TensorFlow将根据选择的学习方案对网络参数进行优化更新。权重和偏置更新完毕后，下一批采样数据将再次送入网络并重复这一过程。这一过程将一直持续至所有批次的数据都已经送入网络。所有的批次构成的一个完整训练过程被称为一个epoch。

当达到训练批次数或者用户指定的标准之后，网络的训练停止。

In [16]:
!mkdir img

子目录或文件 img 已经存在。


In [32]:
# 定义会话
sess = tf.Session()
# 运行初始化器
sess.run(tf.global_variables_initializer())
epochs = 10
batch_size = 256
for i in range(epochs):
    suffle = np.random.permutation(np.arange(len(Y)))
    xtrain = X[suffle]
    ytrain = Y[suffle]
    # minbatch 训练
    for j in range(len(Y)):
        start = i * batch_size
        end = start + batch_size
        sess.run(train, feed_dict={x:xtrain[start:end], y:ytrain[start:end]})
    
    final = sess.run(loss, feed_dict={x:X_test, y:Y_test})
    print(final)

6031023.5
6031023.5
6031023.5
6031023.5
6031023.5
6031023.5
6031023.5
6031023.5
6031023.5
6031023.5


这里再给出一些可以进一步提升结果的方法：规划网络层数和神经元个数，选择不同的初始化和激活方案，引入神经元的dropout层，early stopping等等。除此之外，换用其他类型的深度学习模型，比方说RNN也许可以在任务上达到更优的性能。